In [1]:
import os
parent_path = os.path.dirname(os.getcwd())
os.chdir(parent_path)

In [2]:
from utils.dataloader import GraphTextDataset, GraphDataset, TextDataset
from sklearn.metrics import silhouette_score
from torch_geometric.data import DataLoader
# from models.Model import BaseModel
from models.model2 import GAT_MLP
from models.model3_transfert_learning import GAT_MLP_TL
import numpy as np
from transformers import AutoTokenizer
import torch
from torch import optim
import time
import pandas as pd
from torch.optim.lr_scheduler import ReduceLROnPlateau
from utils.variables import ROOT_DIR
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors

torch.cuda.empty_cache()

In [3]:
# train = pd.read_csv(ROOT_DIR + '/data/train.tsv', sep='\t', header=None)
# sample_train = train.sample(6000, random_state=42)
# sample_train.to_csv(ROOT_DIR + '/data/sample_train.tsv', sep='\t', header=False, index=False)

model_name = 'allenai/scibert_scivocab_uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
gt = np.load("./data/token_embedding_dict.npy", allow_pickle=True)[()]
# val_dataset = GraphTextDataset(root='./data/', gt=gt, split='val', tokenizer=tokenizer)
train_dataset = GraphTextDataset(root='./data/', gt=gt, split='sample_train', tokenizer=tokenizer)
test_dataset = GraphTextDataset(root='./data/', gt=gt, split='new_test', tokenizer=tokenizer)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# checkpoint = torch.load(ROOT_DIR + '/logs/model_20241127_171613.pt')
# checkpoint = torch.load(ROOT_DIR + '/logs/model_20241203_011950.pt') # GAT MLP 1024
# checkpoint = torch.load(ROOT_DIR + '/logs/model_20241202_232208_MLPTL_256.pt') # GAT MLP 256
checkpoint = torch.load(ROOT_DIR + '/logs/model_20241203_041408_PCA.pt') # GAT MLP 256 PCA
# model = GAT_MLP(model_name=model_name, num_node_features=300, nout=768, nhid=300, graph_hidden_channels=300) 
# model = GAT_MLP_TL(model_name=model_name, num_node_features=300, nout=256, nhid=300, graph_hidden_channels=300)
model = GAT_MLP(model_name=model_name, num_node_features=300, nout=256, nhid=300, graph_hidden_channels=300) # GAT MLP PCA 256
model.load_state_dict(checkpoint)
model.to(device)

/tmp/ipykernel_1979308/1861394936.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ROOT_DIR + '/logs/model_20241203_041408_PCA.pt') # GAT MLP 256 

GAT_MLP(
  (graph_encoder): GraphEncoder(
    (relu): ReLU()
    (ln): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
    (conv1): GATConv(300, 300, heads=1)
    (conv2): GATConv(300, 300, heads=1)
    (conv3): GATConv(300, 300, heads=1)
    (mol_hidden1): Linear(in_features=300, out_features=300, bias=True)
    (mol_hidden2): Linear(in_features=300, out_features=256, bias=True)
  )
  (text_encoder): TextEncoder(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(31090, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_featur

In [5]:
### getting U, S, V
print("Calculating PCA...")
pca_loader = DataLoader(train_dataset, batch_size=1)
text_embeddings = []
torch.cuda.empty_cache()
for batch in iter(pca_loader):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)    
    text_embeddings.append(model.get_text_encoder()(input_ids, attention_mask).detach().cpu().numpy())
    input_ids.detach()

    attention_mask.detach()

    with torch.cuda.device(device):
        torch.cuda.empty_cache()
embedding_output = torch.tensor(np.array(text_embeddings).reshape(-1, 768))
U, S, V = torch.pca_lowrank(embedding_output, q=256)
U = U.to(device)
S = S.to(device)
V = V.to(device)
print("PCA done!")
###

Calculating PCA...


/Data/victor.zhuang/Code/MIT/env/lib64/python3.9/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
/Data/victor.zhuang/Code/Molecule Retrieval/utils/dataloader.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case wh

PCA done!


In [6]:
def projected_embeddings(A):
    return torch.matmul(A, V)

In [7]:
def MRR(logits, labels):
    sorted_indices = torch.argsort(logits, descending=True)
    ranks = torch.zeros_like(labels, dtype=torch.float32)
    for i, label in enumerate(labels):
            rank = (sorted_indices[i] == label).nonzero(as_tuple=True)[0].item() + 1
            ranks[i] = 1.0 / rank  
    mrr = ranks.mean()
    return mrr.tolist()

In [10]:
data_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

model.eval()
L = []
for batch in data_loader:
    input_ids = batch.input_ids
    batch.pop('input_ids')
    attention_mask = batch.attention_mask
    batch.pop('attention_mask')
    graph_batch = batch
    
    x_graph, x_text = model(graph_batch.to(device), 
                            input_ids.to(device), 
                            attention_mask.to(device))
    
    x_text = projected_embeddings(x_text)
    
    logits = torch.matmul(x_graph, torch.transpose(x_text, 0, 1))
    labels = torch.arange(logits.shape[0], device=x_graph.device)
    mrr = MRR(logits, labels)

    # GPU memory management
    input_ids.detach()
    attention_mask.detach()
    graph_batch.detach()
    x_graph.detach()
    x_text.detach()

    L.append(mrr)

In [11]:
np.mean(L)

0.13936140753615361

batch: 16, 32, 64, 96

GAT-768: 0.97, 0.96, 0.94, NaN

GAT-MLP 1024: 0.92, 0.86, 0.78, 0.73 

GAT-MLP 256: 0.89, 0.81, 0.71, 0.66

GAT-MLP PCA: 0.23, 0.13